In [69]:
!pip install transformers

In [26]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report, accuracy_score

In [27]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\neera\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\neera\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\neera\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
df = pd.read_csv(r"C:\ML\db\archive_job\job_applicant_dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job Applicant Name  10000 non-null  object
 1   Age                 10000 non-null  int64 
 2   Gender              10000 non-null  object
 3   Race                10000 non-null  object
 4   Ethnicity           10000 non-null  object
 5   Resume              10000 non-null  object
 6   Job Roles           10000 non-null  object
 7   Job Description     10000 non-null  object
 8   Best Match          10000 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 703.3+ KB


In [29]:
df.head(5)

,Job Applicant Name,Age,Gender,Race,Ethnicity,Resume,Job Roles,Job Description,Best Match
0,Daisuke Mori,29,Male,Mongoloid/Asian,Vietnamese,"Proficient in Injury Prevention, Motivation, N...",Fitness Coach,A Fitness Coach is responsible for helping cl...,0
1,Taichi Shimizu,31,Male,Mongoloid/Asian,Filipino,"Proficient in Healthcare, Pharmacology, Medica...",Physician,"Diagnose and treat illnesses, prescribe medica...",0
2,Sarah Martin,46,Female,White/Caucasian,Dutch,"Proficient in Forecasting, Financial Modelling...",Financial Analyst,"As a Financial Analyst, you will be responsibl...",0
3,Keith Hughes,43,Male,Negroid/Black,Caribbean,"Proficient in Budgeting, Supply Chain Optimiza...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1
4,James Davis,49,Male,White/Caucasian,English,"Proficient in Logistics, Negotiation, Procurem...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1


In [30]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [31]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [32]:
# MAX_LEN = 200
# def pad_sequence(seq):
#     seq = seq.split() if isinstance(seq, str) else seq
#     if len(seq) < MAX_LEN:
#         return seq + [0]*(MAX_LEN - len(seq))
#     else:
#         return seq[:MAX_LEN]

In [33]:
df['Resume_clean'] = df['Resume'].apply(preprocess)
# df['Resume_clean'] = df['Resume_clean'].apply(pad_sequence)
df['Job_Description_clean'] = df['Job Description'].apply(preprocess)
# df['Job_Description_clean'] = df['Job_Description_clean'].apply(pad_sequence)

In [34]:
df['combined_text'] = df['Resume_clean'] + "[SEP]" + df['Job_Description_clean']

In [35]:
# X = df['combined_text']
# y = df['Best Match']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [86]:
X_train

9254    proficient problem solving marketing logistics...
1561    proficient navigation safety protocol communic...
1670    proficient client relation contract law negoti...
6087    proficient problem solving scheduling blueprin...
6669    proficient urban design community engagement u...
                              ...                        
5734    proficient patient care medical terminology ph...
5191    proficient scheduling risk management contract...
5390    proficient power bi data visualization python ...
860     proficient social medium content creation writ...
7270    proficient automation problem solving prototyp...
Name: combined_text, Length: 8000, dtype: object

In [36]:
vectorizer = TfidfVectorizer(max_features = 10000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [51]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train_vec, y_train)
rf_preds = rf.predict(X_test_vec)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print(classification_report(y_test, rf_preds))

Random Forest Accuracy: 0.5195
              precision    recall  f1-score   support

           0       0.53      0.53      0.53      1023
           1       0.51      0.51      0.51       977

    accuracy                           0.52      2000
   macro avg       0.52      0.52      0.52      2000
weighted avg       0.52      0.52      0.52      2000

